读取Fashion MINST数据

In [16]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

batch_size = 64

training_data = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Report split sizes
print('Training set has {} instances'.format(len(training_data)))
print('Validation set has {} instances'.format(len(test_data)))

Training set has 60000 instances
Validation set has 10000 instances


LeNet-5 

In [19]:
import torch.nn as nn

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
            nn.Linear(120, 84), nn.Sigmoid(),
            nn.Linear(84, 10))

    def forward(self, x):
        x = self.net(x)
        return x

模型训练

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet().to(device)

epochs = 10

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.9)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.351736  [   64/60000]
loss: 2.288934  [ 6464/60000]
loss: 2.311120  [12864/60000]
loss: 2.326550  [19264/60000]
loss: 2.294600  [25664/60000]
loss: 2.297251  [32064/60000]
loss: 2.296643  [38464/60000]
loss: 2.309734  [44864/60000]
loss: 2.290174  [51264/60000]
loss: 2.303752  [57664/60000]
Test Error: 
 Accuracy: 10.0%, Avg loss: 2.286048 

Epoch 2
-------------------------------
loss: 2.297568  [   64/60000]
loss: 1.960693  [ 6464/60000]
loss: 1.713833  [12864/60000]
loss: 1.419917  [19264/60000]
loss: 1.014244  [25664/60000]
loss: 1.030586  [32064/60000]
loss: 0.944143  [38464/60000]
loss: 0.911985  [44864/60000]
loss: 0.591872  [51264/60000]
loss: 1.057124  [57664/60000]
Test Error: 
 Accuracy: 65.6%, Avg loss: 0.893366 

Epoch 3
-------------------------------
loss: 0.662422  [   64/60000]
loss: 0.794277  [ 6464/60000]
loss: 0.835073  [12864/60000]
loss: 0.692554  [19264/60000]
loss: 0.669855  [25664/60000]
loss: 0.657540  [32064/600

AlexNet

In [28]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),  # 根据输入尺寸计算
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

model = AlexNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 更改数据处理方式
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

training_data = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303494  [   64/60000]
loss: 1.070133  [ 6464/60000]
loss: 0.367762  [12864/60000]
loss: 0.614635  [19264/60000]
loss: 0.588809  [25664/60000]
loss: 0.412316  [32064/60000]
loss: 0.375863  [38464/60000]
loss: 0.725765  [44864/60000]
loss: 0.383158  [51264/60000]
loss: 0.298515  [57664/60000]
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.374812 

Epoch 2
-------------------------------
loss: 0.395371  [   64/60000]
loss: 0.444617  [ 6464/60000]
loss: 0.317055  [12864/60000]
loss: 0.355231  [19264/60000]
loss: 0.204816  [25664/60000]
loss: 0.339073  [32064/60000]
loss: 0.254414  [38464/60000]
loss: 0.308523  [44864/60000]
loss: 0.357969  [51264/60000]
loss: 0.347962  [57664/60000]
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.319386 

Epoch 3
-------------------------------
loss: 0.348927  [   64/60000]
loss: 0.301652  [ 6464/60000]
loss: 0.241173  [12864/60000]
loss: 0.365154  [19264/60000]
loss: 0.159197  [25664/60000]
loss: 0.459029  [32064/600